In [68]:
from urllib.request import urlopen
import xml.etree.ElementTree as et
from pprint import pprint
import uuid
import time
import os

from json import dumps
from kafka import KafkaProducer

In [70]:
TOPIC_NAME = 'repubblica_cronaca'
DATETIME_FORMAT = "%a, %d %b %Y %H:%M:%S %z"
TIME_ZONE = datetime.timezone(datetime.timedelta(0, 7200))

In [66]:
#!rm last.update

In [51]:
def touch(fname):
    open(fname, 'a').close()
    os.utime(fname, None)

In [58]:
def last_update(lock_file = 'last.update'):

    try:
        last_update = os.path.getmtime('last.update')
    except:
        last_update = 0

    return datetime.datetime.fromtimestamp(last_update, tz=TIME_ZONE)

In [60]:
last_update()

datetime.datetime(1970, 1, 1, 2, 0, tzinfo=datetime.timezone(datetime.timedelta(0, 7200)))

In [69]:
# Connect to the Kafka
producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))

NoBrokersAvailable: NoBrokersAvailable

In [65]:
while True:
    
    RSS_URL = 'http://www.repubblica.it/rss/cronaca/rss2.0.xml?ref=RHFT'

    rss=urlopen(RSS_URL)

    tree = et.parse(rss)

    root = tree.getroot()

    count = 0
    
    """ 
    Loop over the item in the RSS, formate the message in JSON format,
    check if the message is more recent than the last update and if so
    send the message to the Kafka server
    """
    for item in root.getiterator(tag='item'):
    
        item = {
            'title'       : item.find('title').text,
            'link'        : item.find('link').text,
            'author'      : item.find('author').text,
            'category'    : item.find('category').text,
            'pubDate'     : item.find('pubDate').text,
            'description' : item.find('description').text,
            'guid'        : item.find('guid').text,
            'uuid'        : uuid.uuid3(uuid.NAMESPACE_URL, item.find('link').text)
        }
    
        pubDate =  datetime.datetime.strptime(item['pubDate'], DATETIME_FORMAT)
    
        # Skip if publication date is earlier than the last update
        # i.e. the item have been already processed.
        if pubDate < last_update():
            continue
    
        count += 1
    
        # Send the message to the Kafka server
        producer.send('TOPIC_NAME', value=item)
        #pprint(item)
    
    print('Update: %d new items since %s' % (count, last_update()))
    
    touch('last.update')
    
    time.sleep(60)

{'author': 'repubblicawww@repubblica.it (Redazione Repubblica.it)',
 'category': 'cronaca',
 'description': '<p><a '
                'href="https://bari.repubblica.it/cronaca/2019/07/09/news/la_taranta_e_la_pizzica_verso_il_riconoscimento_unesco_concertone_con_elisa_e_gue_pequeno-230791897/?rssimage"> '
                '<img '
                'src="https://www.repstatic.it/content/localirep/img/rep-bari/2019/07/09/140630829-6d509f7f-fa33-47be-8dc2-d1b948208255.jpg" '
                'width="140" align="left" hspace="10"></a>Annunciata la '
                'candidatura della Taranta e della pizzica a patrimonio '
                "immateriale dell'umanità dell'Unesco. L'evento del 24 agosto "
                'in diretta su Rai2</p>',
 'guid': 'https://bari.repubblica.it/cronaca/2019/07/09/news/la_taranta_e_la_pizzica_verso_il_riconoscimento_unesco_concertone_con_elisa_e_gue_pequeno-230791897/?rss',
 'link': 'https://bari.repubblica.it/cronaca/2019/07/09/news/la_taranta_e_la_pizzica_verso

KeyboardInterrupt: 